# Convolve EEG
This Jupyter Notebook takes the imported `.npz` files containing the clean and artifact datasets and convolves the appropriate channels. The convolved data can be saved as a `.npz` file.

The workflow is as follows:
1. Import RAW datasets
    - This will import the clean data as well as the eye movement (`eye`) and muscle artifact (`mus`) data.
2. Combine the data
    - Select `eye` or `mus` data to be convoluted with the clean data.
3. Plot the clean and convolved power spectral density (PSD). This section is **optional** .
4. Save the data in a `.npz` or `.mat` format for further analysis.The saved data contains the following variables:
    - `clean`: Clean EEG data, organized in a tensor of samples $\times$ channels $\times$ ssvep stimulus [V].
        The SSVEP stimulus are: 10, 12, and 15 Hz.
    - `conv`: Convoluted clean and `eye` or `mus` artifact. The type of artifact is shown in the file name. The data is organized in the same fashion as `clean`.
    - `clean_chans`: List of the channel names of the clean data (64)
    - `chans`: List of the channels names in the 10-20 format (18). This number is smaller than `clean chans` because the Temple dataset has fewer channels.
    - `srate`: Sampling rate of the data [Hz].

In [16]:
# Import libraries
import os
import numpy as np
import scipy.signal as signal
import scipy.io
import matplotlib.pyplot as plt

# Determine working directory
files_path = os.getcwd()

## Import RAW datasets
First, we will import the BETA and Temple University datasets in the `.npz` format

**Note**: Datasets are in [V]. However, the plots are scaled to [$\mu$ V].

In [17]:
# Settings
plot_raw = False # Boolean to plot raw data after imported
ssvep_plot = 0  # SSVEP stimulus to plot
                # - 0 = 10 Hz
                # - 1 = 12 Hz
                # - 2 = 15 Hz
artifact_plot = 0   # Index of artifact to plot
                    # Note: eye and muscle artifacts might have different number of indices
beta_file = '\\Data\Imported\S1.npz'                # Relative path to BETA file
temple_file = '\\Data\\Imported\\artifact_data.npz' # Relative path to Temple University file

# Import data
clean = np.load(files_path+beta_file, allow_pickle=True) 
artifact = np.load(files_path+temple_file, allow_pickle=True) 

# Check that sampling rates are the same
if clean['srate'] == artifact['srate']:
    srate = clean['srate']
else:
    print('Warning sampling rates of clean and artifact data are not the same')
    print(f'Clean =',clean['srate'])
    print(f'Artifact = '+artifact['srate'])

# Print raw data if desired
if plot_raw:
    plot_clean = clean['eeg'][:,:,ssvep_plot]*1e6
    time_eeg = np.linspace(0, len(plot_clean)/clean['srate'], len(plot_clean))

    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_title('RAW | clean data')
    ax.set_xlabel('Time [sec]')
    ax.set_ylabel('Amplitude [$\mu$V]')
    ax.plot(time_eeg, plot_clean)
    # ax.plot(time_eeg, clean_eeg[:,ic_chans,2])
    ax.grid()
    plt.show()

    plot_eye = artifact['eye_eeg'][artifact_plot]*1e6
    plot_mus = artifact['mus_eeg'][artifact_plot]*1e6
    time_eye = np.linspace(0, len(plot_eye)/artifact['srate'], len(plot_eye))
    time_mus = np.linspace(0, len(plot_mus)/artifact['srate'], len(plot_mus))

    [fig, axs] = plt.subplots(2,1)
    axs[0].set_title('RAW | Artifact data')
    axs[0].set_ylabel('Eye [$\mu$V]')
    axs[0].grid()
    axs[0].plot(time_eye, plot_eye)
    axs[1].set_ylabel('Muscle [$\mu$V]')
    axs[1].set_xlabel('Time [sec]')
    axs[1].grid()
    axs[1].plot(time_mus, plot_mus)
    plt.show()

## Combine data
This section combines the data from the clean trials with the artifact data. The clean data has **64** EEG channels while the artifact data has **15** EEG channels. This section of the code makes sure that you are using the same channels in both datasets

In [18]:
# Settings
plot_conv = False   # Boolean to plot the convolved data
select_artifact = 'eye'     # Select type of artifact -> Options = 'eye', or 'mus'
select_artifact_block = 3   # Select block of artifact

# Find sorted intersection of channels
# - Setup channels to find intersection
artifact_chans = artifact['chans'][0:19] # Order of selected channels to use [n=19]
clean_chans = clean['chans'][:,3] # Order of clean channels [n=64]  

# - Find channel intersection
# -- chans = combined channels
# -- ic_chans = Indexes of clean channels
# -- ia_chans = Indexes of artifact channels
[chans, ic_chans, ia_chans] = np.intersect1d(clean_chans, artifact_chans, return_indices=True)  
print(f'Channels kept: {chans}')

# For each ssvep stimulus 
# Note: Only the intersecting channels are selected
conv_data = np.empty(len(clean['ssvep']), dtype='object')
time_conv = np.empty(len(clean['ssvep']), dtype='object')

for [i,ssvep] in np.ndenumerate(clean['ssvep']):
    temp_data = np.squeeze(clean['eeg'][:,ic_chans,i])
    
    if select_artifact == 'eye':
        temp_artifact = artifact['eye_eeg'][select_artifact_block][:,ia_chans]
    elif select_artifact == 'mus':
        temp_artifact = artifact['mus_eeg'][select_artifact_block][:,ia_chans]
    else:
        print('Warning, wrong type of artifact selected')

    # Convolve data
    conv_data[i] = signal.convolve(temp_data, temp_artifact, mode='same', method='fft')
    # conv_data[i] = signal.convolve(temp_data, temp_artifact, mode='full')
    time_conv[i] = np.linspace(0, len(conv_data[i])/srate, len(conv_data[i]))

    if plot_conv:
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.set_xlabel('Time [sec]')
        ax.set_ylabel(f'SSVEP - {ssvep} Hz [$\mu$V]')
        ax.grid()
        plt.plot(time_conv[i], conv_data[i]*1e6)
        plt.show()

Channels kept: ['C3' 'C4' 'CZ' 'F3' 'F4' 'F7' 'F8' 'FP1' 'FP2' 'FZ' 'O1' 'O2' 'P3' 'P4'
 'PZ']


In [19]:
np.shape(conv_data[0])

(750, 15)

## Plot PSD
This section plots the power spectral density (PSD) of the clean and convoluted (EEG + artifact) data.

In [20]:
plot_psd = False # Boolean to plot the PSD of the data

if plot_psd:
    for [i,ssvep] in np.ndenumerate(clean['ssvep']):     
        # - Clean data
        eeg_temp = np.squeeze(clean['eeg'][:,:,i]) # Enable for clean data
        clean_win = np.ones(len(eeg_temp))
        [f_clean, pxx_clean] = signal.welch(eeg_temp, fs=srate, window=clean_win, axis=0)
        
        # - Convoluted data
        data = conv_data[i]
        win = np.ones(len(data))
        [f, pxx] = signal.welch(data, fs=srate, window=win, axis=0)

        fig, ax = plt.subplots(2,1)
        ax[0].set_xlabel('Frequency [Hz]')
        ax[0].set_ylabel('Clean PSD\n[V$^2$/Hz]')
        ax[0].set_xlim(5, 35)
        ax[0].set_title(f'SSVEP - {ssvep} Hz')
        ax[0].grid()
        ax[0].plot(f_clean, pxx_clean)

        ax[1].set_ylabel('Convolved PSD\n[V$^2$/Hz]')
        ax[1].set_xlim(5, 35)
        ax[1].grid()
        ax[1].plot(f, pxx)

        plt.tight_layout()
        plt.show()

## Save data
Save clean and convolved data to the same file, include the channels and sampling rate [Hz]

In [21]:
save_data = True   # Boolean to save convolved data
save_matlab = True  # Boolean to save Matlab formatted data
save_loc = 'Data\\Convolved'
save_name = f'conv_{select_artifact}_data'

# Determine parent directory
par_dir = os.getcwd()

# Check if save_loc exist. If it doesn't exist, create it        
if os.path.isdir(os.path.join(par_dir, save_loc)):
    pass
else:
    os.mkdir(os.path.join(par_dir, save_loc))

# Concatenate full file name
full_file_name = os.path.join(par_dir,save_loc,save_name)

if save_data:   
    np.savez(full_file_name, clean=clean['eeg'], conv=conv_data, \
        clean_chans=clean_chans, chans=chans, srate=srate)

if save_matlab:
    data_dict = {'clean':clean['eeg'], 'conv':conv_data, 'clean_chans':clean_chans, \
        'chans':chans, 'srate':srate}
    scipy.io.savemat(full_file_name+'.mat', data_dict)